## Required packages

In [1]:
# - tensorflow
# - keras
# - python_speech_features
# - imblearn 
# - Librosa

In [2]:
import numpy
import os

## train set extractor

In [ ]:
def csv_extractor(file_name):

    csv_open = open(file_name,"r")
    feature_names = []
    labels = []

    for index, wave in enumerate(csv_open):
        wave = wave.strip("\n")
        wave = wave.split(",")
    
        feature = wave[0]
        label = wave[1]
        if index == 0:
            continue
    
        else:
            feature_names.append(feature)
            labels.append(label)

    feature_names = numpy.array(feature_names)
    labels = numpy.array(labels)
    
    return feature_names, labels
CSV_TRAIN = csv_extractor("train.csv")

# test set extractor

In [ ]:
def csv_extractor_test(file_name):

    csv_open = open(file_name,"r")
    
    feature_name_test = []

    for index, wave in enumerate(csv_open):
        wave = wave.strip("\n")

        feature = wave
        if index == 0:
            continue
    
        else:
            print(index,feature)
            feature_name_test.append(feature)

    feature_name_test = numpy.array(feature_name_test)
    
    return feature_name_test

CSV_TEST = csv_extractor_test("test.csv")

# Store train and test in an array 

In [ ]:
feature_names, labels = CSV_TRAIN 
feature_name_test = CSV_TEST

## checkpoint

In [ ]:
print("len of train(features) examples:", feature_names.shape)
print("len of test examples",feature_name_test.shape)
print()
print("example of a file name:", feature_names[-1])
print("example of a label:", labels[-1])

# step 1: Open our Dataset & Explore the structure of wave samples
- Okay, now that we have a list with all wave names and labels.
<br>
- Load wave file and visualize its waveform (using librosa)


In [ ]:
my_path = 'C:/Users/voice/Downloads/dataset/wav/'

import librosa
import librosa.display
import matplotlib.pyplot as plt

data, sampling_rate = librosa.load(my_path+'00000b653e96c0697676f91ec9dfb9fc9b4d085f.wav')

plt.figure(figsize=(15, 5))
librosa.display.waveplot(data, sr=sampling_rate)

# step 2: match feature_names with wave_files and extract MFCC features

- all_wave = folder with all wave files
- feature_names(numpy array) = name of all wave files
- labels(numpy array) = labels of all feature_names
- function will return features and labels

In [ ]:
my_path_examples = []
for example in os.listdir(my_path):
    my_path_examples.append(example)
    
my_path_examples = numpy.array(my_path_examples)

## checkpoint

In [ ]:
print("len of train(features) examples:", feature_names.shape)
print("len of train(labels) examples:", labels.shape)
print("len of train + test examples:", my_path_examples.shape)

# MFCC feature extraction (train set)

In [ ]:
from python_speech_features import mfcc
from python_speech_features import logfbank
import scipy.io.wavfile as wav
import librosa

def mfcc_features(my_path,feature_names, labels):

    #mean_mfcc_list = []
    #std_mfcc_list = []
    features = []
    all_labels = []
    
    for index, (wave_name,label) in enumerate(zip(feature_names,labels)):
        
        for wave in os.listdir(my_path): 
            #if wav and wave_name are equal
            #extract mfcc features and append them to features
            #append label to labels
            if wave == wave_name:
                #x,sr = librosa.load(my_path+wave)
                #mean_mfcc = numpy.mean(librosa.feature.mfcc(x,sr,n_mfcc = 40).T, axis = 0)
                #std_mfcc = numpy.std(librosa.feature.mfcc(x,sr,n_mfcc = 40).T, axis = 0)
                (rate,sig) = wav.read(my_path+wave)
                mfcc_feat = mfcc(sig,rate)
                    
                #mean_mfcc_list.append(mean_mfcc)
                #std_mfcc_list.append(std_mfcc)
                
                all_labels.append(label)
                print(index)
                 
    #mean_mfcc_list = numpy.array(mean_mfcc_list) 
    #std_mfcc_list = numpy.array(std_mfcc_list)
    features = numpy.array(features)
    all_labels = numpy.array(all_labels)
    
    return features, all_labels

MFCC_FEATURES = mfcc_features(my_path, feature_names, labels)

# MFCC feature extraction (test set)

In [ ]:
from python_speech_features import mfcc
from python_speech_features import logfbank
import scipy.io.wavfile as wav

def mfcc_features_test(my_path,feature_names):

    all_features = []
    all_filename_test = []
    
    for index, wave_name in enumerate(feature_names):
        
        for wave in os.listdir(my_path): 
            #if wav and wave_name are equal
            #extract mfcc features and append them to features
            if wave == wave_name: 
                (rate,sig) = wav.read(my_path+wave)
                mfcc_feat = mfcc(sig,rate, numcep=40)
                all_filename_test.append(wave)
                all_features.append(mfcc_feat)
                print(index)
                 
    all_features = numpy.array(all_features) 
    all_filename_test = numpy.array(all_filename_test)
    
    return all_features, all_filename_test

MFCC_FEATURES_test = mfcc_features_test(my_path, feature_name_test)


## extract features and labels for train and test

In [ ]:
MFCC_features, MFCC_labels = MFCC_FEATURES
MFCC_features_test, all_labels_test = MFCC_FEATURES_test

## checkpoint

In [ ]:
print("shape of extracted mfcc features from test set:", MFCC_features_test.shape)
print("shape of extracted mfcc features:", MFCC_features[0].shape)
print("shape of extracted mfcc labels:", MFCC_labels.shape)
print("shape of extracted filter bank energy from test set:", LFBE_features_test.shape)
print("shape of extracted log filter bank energy features:", LFBE_features[0].shape)
print("shape of extracted log filter bank energy labels:", LFBE_labels.shape)

## zero pad MFCC features train set

In [ ]:
max_shape = 0
for x in MFCC_features:
    all_shapes = x.shape[0]
    if all_shapes > max_shape:
        max_shape = all_shapes

In [ ]:
new_spoken_train = []
for x in MFCC_features:
    difference = max_shape-x.shape[0]
    zero_padded = numpy.pad(x,((0,difference),(0,0)), "constant")
    new_spoken_train.append(zero_padded)

new_spoken_train = numpy.array(new_spoken_train)

# zero pad MFCC features test set

In [ ]:
max_shape_test = 0
for x in MFCC_features_test:
    all_shapes = x.shape[0]
    if all_shapes > max_shape_test:
        max_shape_test = all_shapes

In [ ]:
new_spoken_test = []
for x in MFCC_features_test:
    difference = max_shape_test-x.shape[0]
    zero_padded = numpy.pad(x,((0,difference),(0,0)), "constant")
    new_spoken_test.append(zero_padded)

new_spoken_test = numpy.array(new_spoken_test)

## checkpoint

In [ ]:
new_spoken_train.shape

In [ ]:
new_spoken_test.shape

# Apply PCA

In [ ]:
from sklearn.decomposition import PCA
#Fitting the PCA algorithm with our Data
pca_train = PCA().fit(new_spoken_train)
#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(numpy.cumsum(pca_train.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('MFCC Explained Variance')
plt.show()

# Apply oversampling

In [ ]:
new_spoken_train = numpy.reshape(new_spoken_train,(94824,1287))

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(new_spoken_train)
new_spoken_train = scaler.fit_transform(new_spoken_train)
new_spoken_test = scaler.transform(new_spoken_test)

# Apply oversampling

In [ ]:
from imblearn.over_sampling import SMOTE

ros = SMOTE(random_state=42)
new_mfcc_features,  mfcc_labels = ros.fit_resample(new_spoken_train, MFCC_labels)

dictionary_2 = {}
for i in mfcc_labels:
    dictionary_2[i] = dictionary_2.get(i,0) + 1

print(dictionary_2)

# reshape 

In [ ]:
new_mfcc_features = numpy.reshape(new_mfcc_features,(new_mfcc_features.shape[0],99,13))

# data overview

In [ ]:
print(new_spoken_test.shape)
print(new_mfcc_features.shape)
print(mfcc_labels.shape)

# reshape

In [ ]:
new_spoken_test = numpy.reshape(new_spoken_test,(11005,1287))
new_spoken_train = numpy.reshape(new_mfcc_features,(127190,1287))

# One-hot encode Labels

In [ ]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = mfcc_labels
values = array(data)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
#print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
new_mfcc_labels = onehot_encoded

# import train_test_split and split data in train & val

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_spoken_train, new_mfcc_labels, test_size=0.20, 
                                                    random_state=42, stratify = new_mfcc_labels, shuffle= True)

## checkpoint

In [ ]:
print(X_train.shape)
print(X_test.shape)

## reshape

In [ ]:
X_train = numpy.reshape(X_train,(X_train.shape[0],99,13))
X_test = numpy.reshape(X_test,(X_test.shape[0],99,13))
print(X_train.shape)
print(X_test.shape)

## checkpoint

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:",y_train.shape)
print("X_test shape:",X_test.shape)
print("y_test shape:",y_test.shape)

# Build classification model: 1D CNN

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from wandb import magic
model = Sequential()
model.add(Conv1D(128, 2, activation = "relu", input_shape=(99,13)))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(256, 2, activation = "relu"))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Dropout(0.3))
model.add(Conv1D(356, 2, activation = "relu"))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(456, 2, activation = "relu"))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(1048))
model.add(Dense(35, activation='softmax'))

In [ ]:
model.summary()

## train model

In [ ]:
from keras.optimizers import Adam

model.compile(loss= "categorical_crossentropy",
              optimizer= Adam(lr=0.0001),
              metrics=['accuracy'])

cnnhistory= model.fit(X_train, y_train,batch_size= 64, epochs= 150 ,validation_data=(X_test, y_test))

## result after training

In [ ]:
result = model.evaluate(X_test, y_test)
print("validation accuracy:", result[1])

# CNN prediction output

In [ ]:
new_spoken_test = numpy.reshape(new_spoken_test,(11005,99,13,1))

In [ ]:
file_open = open("result.csv","a+")
for index in range(11005):
    file_name = all_labels_test[index]
    predicted_word = label_encoder.inverse_transform([argmax(model.predict(new_spoken_test)[index, :])])
    predicted_word = "'".join(word)
    file_open.write(file_name+","+predicted_word+"\n")
    
file_open.close()